Constructing the company dataset, joining moneyball 1.1 successful and failed companies + scraping linkedin

In [280]:
import pandas as pd
import openpyxl
import pickle
import sys
import json

In [281]:
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


Successful Companies

In [282]:
s_1 = pd.read_excel("/content/gdrive/MyDrive/Colab Notebooks/weave/Datasets/moneyball_success.xlsx", sheet_name = 'Company')
s_2 = pd.read_excel("/content/gdrive/MyDrive/Colab Notebooks/weave/Datasets/moneyball_success.xlsx", sheet_name = 'Long company descriptions')
s_3 = pd.read_excel("/content/gdrive/MyDrive/Colab Notebooks/weave/Datasets/moneyball_success.xlsx", sheet_name = 'Funding rounds')
s_4 = pd.read_excel("/content/gdrive/MyDrive/Colab Notebooks/weave/Datasets/moneyball_success.xlsx", sheet_name = 'Founder Linkedin URLs')
weave_successful = []

In [283]:
df_s = pd.merge(s_1, s_2, on = ['org_uuid', 'org_name']).drop(columns = 'short_description')
df_s = pd.merge(df_s, s_3, on = ['org_uuid', 'org_name'])
df_s = pd.merge(df_s, s_4, on = ['org_uuid', 'org_name'])
df_s['founded_on'] = [c.strftime("%Y") for c in df_s['founded_on']]
df_s.dropna(inplace = True)
df_s = df_s[df_s['raised_amount_usd'] > 0]

In [284]:
df_test = pd.merge(s_1, s_2, on = ['org_uuid', 'org_name']).drop(columns = 'short_description')
df_test = pd.merge(df_test, s_3, on = ['org_uuid', 'org_name'])
df_test.loc[0:10,:]

,org_uuid,org_name,domain,status,founded_on,category_list,category_groups_list,country_code,city,long_description,investment_type,round_created_at,raised_amount_usd,investor_count,investors_names,investors_uuids
0,a2323b6c-29b9-4750-905c-cdcbd9ce92b8,10X Banking,10xbanking.com,operating,2016-01-01,"Banking,Financial Services,FinTech,Software","Financial Services,Lending and Investments,Sof...",GBR,London,"10x Banking (10x), a financial services techno...",series_a,2017-09-21 01:07:18.000000 UTC,45867909.0,2.0,"Oliver Wyman,Ping An Global Voyager","88213ccc-71f8-1024-72cf-44e863af4993,c8cb5641-..."
1,cf3b00cf-ffbf-0e4f-6427-a60e5060de39,10X Genomics,10xgenomics.com,ipo,2012-01-01,"Biotechnology,Genetics,Health Care,Medical","Biotechnology,Health Care,Science and Engineering",USA,Pleasanton,10x Genomics is creating revolutionary DNA seq...,seed,2018-04-27 04:18:33.000000 UTC,4700000.0,1.0,Silicon Valley Bank,4eee5e60-3b06-6fe8-cc2c-29119a96051d
2,0680987f-23cd-4fa5-850c-38326fffaa36,"111, Inc.",corporate.111.com.cn,ipo,2010-01-01,"Health Care,Medical",Health Care,CHN,Shanghai,"111,Inc. is a leading digital and mobile healt...",series_a,2018-09-14 07:49:39.000000 UTC,2100000.0,1.0,Ivy Capital,f8eed790-8c07-4e49-c873-0f7245e393ff
3,a79a6ae8-3c74-0db7-7e6b-b4f7662d37e8,17zuoye,ucenter.17zuoye.com,ipo,2011-01-01,"E-Learning,EdTech,Education,Internet,Language ...","Education,Internet Services,Software",CHN,Shanghai,17zuoye is an online learning platform for K-1...,series_a,2015-02-20 15:29:12.000000 UTC,5000000.0,1.0,Shunwei Capital,24540ba2-35b0-a757-f75e-87565c50692e
4,a79a6ae8-3c74-0db7-7e6b-b4f7662d37e8,17zuoye,ucenter.17zuoye.com,ipo,2011-01-01,"E-Learning,EdTech,Education,Internet,Language ...","Education,Internet Services,Software",CHN,Shanghai,17zuoye is an online learning platform for K-1...,angel,2015-02-20 15:37:33.000000 UTC,NaN,4.0,"Bob Xiaoping Xu,Eugene Zhang,Victor Wang,ZhenFund","6771e55d-27a3-2b7a-8f58-ba2b3059c4c8,29217ef4-..."
5,8e5b2628-6d2b-4f0a-9808-07c935156515,1inch,1inch.io,operating,2020-07-09,"Blockchain,Cryptocurrency,Information Services...","Financial Services,Information Technology,Othe...",KNA,Old Road Town,1inch Limited builds decentralized protocols a...,seed,2020-08-13 10:38:38.000000 UTC,2800000.0,11.0,"Binance Labs,Divergence Ventures,Dragonfly,FTX...","8b52e412-7d11-4b4d-ae49-3811f29fb05e,660f9563-..."
6,8e5b2628-6d2b-4f0a-9808-07c935156515,1inch,1inch.io,operating,2020-07-09,"Blockchain,Cryptocurrency,Information Services...","Financial Services,Information Technology,Othe...",KNA,Old Road Town,1inch Limited builds decentralized protocols a...,series_a,2020-12-03 07:44:06.000000 UTC,12000000.0,11.0,"Blockchain Capital,Cashican People LLC,Fabric ...","467c04e8-580c-cefa-8d51-f2eb42ab0844,1eefa3e2-..."
7,1ca6a7b3-465b-443d-9028-388cfd1ad060,1Komma5¬∞,1komma5grad.com,operating,2021-01-01,NaN,NaN,DEU,Hamburg,KOMMA5 ¬∞ offers all owners of electric vehicl...,seed,2021-12-06 08:36:38.000000 UTC,NaN,1.0,Porsche Ventures,b88f8432-9309-405e-a723-e53d47dba144
8,1ca6a7b3-465b-443d-9028-388cfd1ad060,1Komma5¬∞,1komma5grad.com,operating,2021-01-01,NaN,NaN,DEU,Hamburg,KOMMA5 ¬∞ offers all owners of electric vehicl...,series_a,2022-06-30 15:45:06.000000 UTC,217540509.0,9.0,"Blue Elephant Capital,Eurazeo,Haniel,Jan Klatt...","fdc5a821-26a9-4f97-80dd-633ebc95e3fe,c250ecc2-..."
9,0995a94f-0661-0db6-fcae-cbb89937c945,360Learning,360learning.com,operating,2013-01-01,"B2B,Corporate Training,E-Learning,EdTech,Human...","Administrative Services,Education,Other,Software",USA,New York,360Learning develops a collaborative learning ...,seed,2020-11-30 18:29:41.000000 UTC,NaN,1.0,Sebastien Breteau,ab7cdecc-cd81-8e4e-f962-f9624747e5b0


In [285]:
df_inv_agg_s = df_s.groupby(['org_name', 'org_uuid'])[['investor_count', 'raised_amount_usd']].sum().reset_index()
df_inv_agg_s['investor_number_normalised']=(df_inv_agg_s['investor_count']-df_inv_agg_s['investor_count'].mean())/df_inv_agg_s['investor_count'].std()
df_inv_agg_s['raised_amount_usd_normalised']=(df_inv_agg_s['raised_amount_usd']-df_inv_agg_s['raised_amount_usd'].mean())/df_inv_agg_s['raised_amount_usd'].std()

In [286]:
investor_number = []
for c in df_inv_agg_s['investor_number_normalised']:
    if c <= 0.1:
        investor_number.append('low')
    elif c >= 0.1 and c < 1.5:
        investor_number.append('mid')
    elif c >= 1.5:
        investor_number.append('high')

df_inv_agg_s['investor_number'] = investor_number
#df_inv_agg_s.drop(columns = ['investor_count','investor_number_normalised'], inplace = True) 

In [287]:
raised_amount = []
for c in df_inv_agg_s['raised_amount_usd_normalised']:
    if c <= 0.1:
        raised_amount.append('low')
    elif c >= 0.1 and c < 2:
        raised_amount.append('mid')
    elif c >= 2:
        raised_amount.append('high')
df_inv_agg_s['investment_size'] = raised_amount


In [288]:
df_s = df_s.merge(df_inv_agg_s[['org_name', 'org_uuid','investor_number', 'investment_size']], how = 'left', on = ['org_name', 'org_uuid'])
df_s.drop(columns = 'category_list', inplace = True)
df_s.rename(columns = {'category_groups_list': 'category_list'}, inplace = True)
# df_s.to_excel(r'/content/gdrive/MyDrive/Colab Notebooks/weave/Datasets/successful_comp.xlsx', index=False)


Failing Companies

In [289]:
fail_1 = pd.read_excel("/content/gdrive/MyDrive/Colab Notebooks/weave/Datasets/moneyball_fail.xlsx", sheet_name = 'Company')
fail_2 = pd.read_excel("/content/gdrive/MyDrive/Colab Notebooks/weave/Datasets/moneyball_fail.xlsx", sheet_name = 'Long company descriptions')
fail_3 = pd.read_excel("/content/gdrive/MyDrive/Colab Notebooks/weave/Datasets/moneyball_fail.xlsx", sheet_name = 'Funding rounds')
fail_4 = pd.read_excel("/content/gdrive/MyDrive/Colab Notebooks/weave/Datasets/moneyball_fail.xlsx", sheet_name = 'Founder Linkedin URLs')
weave_fail = []


In [290]:
df_f = pd.merge(fail_1, fail_2, on = ['org_uuid', 'org_name']).drop(columns = 'short_description')
df_f = pd.merge(df_f, fail_3, on = ['org_uuid', 'org_name'])
df_f = pd.merge(df_f, fail_4, on = ['org_uuid', 'org_name'])
df_f['founded_on'] = [c.strftime("%Y") for c in df_f['founded_on']]
df_f = df_f[df_f['raised_amount_usd'] > 0]
df_f.dropna(inplace = True)

<ipython-input-290-8b40e5784410>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_f.dropna(inplace = True)


In [291]:
df_inv_agg_f = df_f.groupby(['org_name', 'org_uuid'])[['investor_count', 'raised_amount_usd']].sum().reset_index()
df_inv_agg_f['investor_number_normalised']=(df_inv_agg_f['investor_count']-df_inv_agg_f['investor_count'].mean())/df_inv_agg_f['investor_count'].std()
df_inv_agg_f['raised_amount_usd_normalised']=(df_inv_agg_f['raised_amount_usd']-df_inv_agg_f['raised_amount_usd'].mean())/df_inv_agg_f['raised_amount_usd'].std()

In [292]:
investor_number = []
for c in df_inv_agg_f['investor_number_normalised']:
    if c <= 0.1:
        investor_number.append('low')
    elif c >= 0.1 and c < 1.5:
        investor_number.append('mid')
    elif c >= 1.5:
        investor_number.append('high')


df_inv_agg_f['investor_number'] = investor_number
df_inv_agg_f.drop(columns = ['investor_count','investor_number_normalised'], inplace = True)

raised_amount = []
for c in df_inv_agg_f['raised_amount_usd_normalised']:
    if c <= 0.1:
        raised_amount.append('low')
    elif c >= 0.1 and c < 2:
        raised_amount.append('mid')
    elif c >= 2:
        raised_amount.append('high')


df_inv_agg_f['investment_size'] = raised_amount
df_inv_agg_f.drop(columns = ['raised_amount_usd_normalised', 'raised_amount_usd'], inplace = True)




In [293]:
df_f = df_f.merge(df_inv_agg_f, how = 'left', on = ['org_name', 'org_uuid'])
df_f.rename(columns = {'category_groups_list': 'category'}, inplace = True)
df_f = df_f[['org_uuid', 'org_name', 'founded_on',
       'category_list','country_code', 'city', 'long_description',
       'investment_type', 'investors_names', 'investors_uuids',
       'founder_linkedin_url', 'investor_number', 'investment_size']]
# df_f.to_excel(r'/content/gdrive/MyDrive/Colab Notebooks/weave/Datasets/failed_comp.xlsx', index=False)


Investors

In [294]:
inv = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/weave/Datasets/investor_profiles.csv')
inv.dropna(inplace = True)
inv['investment_count_normalised']=(inv['investment_count']-inv['investment_count'].mean())/inv['investment_count'].std()

In [295]:
investment_count = []
for c in inv['investment_count_normalised']:
    if c <= 0.1:
        investment_count.append('D')
    elif c >= 0.1 and c < 1:
        investment_count.append('C')
    elif c >= 1 and c < 10:
        investment_count.append('B')
    elif c >= 10:
        investment_count.append('A')

inv['investment_count'] = investment_count
inv.drop(columns = 'investment_count_normalised', inplace = True)
# inv.to_excel(r'/content/gdrive/MyDrive/Colab Notebooks/weave/Datasets/investor-df.xlsx', index=False)


In [296]:
df_i = pd.read_excel("/content/gdrive/MyDrive/Colab Notebooks/weave/Datasets/investor-df.xlsx")
df_f = pd.read_excel("/content/gdrive/MyDrive/Colab Notebooks/weave/Datasets/failed_comp.xlsx")
df_s = pd.read_excel("/content/gdrive/MyDrive/Colab Notebooks/weave/Datasets/successful_comp.xlsx")

In [297]:
# Add Labels: 1 for success, 0 for fail
df_s['labels'] = 1
df_f['labels'] = 0

Find Founders Info !!!!!!!!

In [298]:
# Load founders profiles
success_enriched_linkedin_profiles = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/weave/Datasets/success_enriched_linkedin_profiles.csv', index_col='linkedin_url')
fail_enriched_linkedin_profiles = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/weave/Datasets/fail_enriched_linkedin_profiles.csv', index_col='linkedin_url')

In [299]:
df_s1 = df_s[['org_uuid','org_name','category_list','labels','investors_names','investors_uuids','founder_linkedin_url']]
df_f1 = df_f[['org_uuid','org_name','category_list','labels','investors_names','investors_uuids','founder_linkedin_url']]

In [300]:
df_s1

,org_uuid,org_name,category_list,labels,investors_names,investors_uuids,founder_linkedin_url
0,a2323b6c-29b9-4750-905c-cdcbd9ce92b8,10X Banking,"Financial Services,Lending and Investments,Sof...",1,"Oliver Wyman,Ping An Global Voyager","88213ccc-71f8-1024-72cf-44e863af4993,c8cb5641-...",https://www.linkedin.com/in/antony-jenkins/
1,cf3b00cf-ffbf-0e4f-6427-a60e5060de39,10X Genomics,"Biotechnology,Health Care,Science and Engineering",1,Silicon Valley Bank,4eee5e60-3b06-6fe8-cc2c-29119a96051d,https://www.linkedin.com/in/ben-hindson-09739813/
2,cf3b00cf-ffbf-0e4f-6427-a60e5060de39,10X Genomics,"Biotechnology,Health Care,Science and Engineering",1,Silicon Valley Bank,4eee5e60-3b06-6fe8-cc2c-29119a96051d,https://www.linkedin.com/in/serge-saxonov-7b42...
3,0680987f-23cd-4fa5-850c-38326fffaa36,"111, Inc.",Health Care,1,Ivy Capital,f8eed790-8c07-4e49-c873-0f7245e393ff,https://www.linkedin.com/in/junling-liu-2b90724/
4,a79a6ae8-3c74-0db7-7e6b-b4f7662d37e8,17zuoye,"Education,Internet Services,Software",1,Shunwei Capital,24540ba2-35b0-a757-f75e-87565c50692e,https://www.linkedin.com/in/dun-xiao-3622629/
...,...,...,...,...,...,...,...
5808,542067a6-bb15-eb5d-b01b-a383bb3df7f2,Zwift,"Health Care,Sports",1,"Alarik Myrin,Eric Min","3f8e4bb0-0ded-06e0-37e4-d43b46f44940,37b701ff-...",https://www.linkedin.com/in/scottbarger/
5809,542067a6-bb15-eb5d-b01b-a383bb3df7f2,Zwift,"Health Care,Sports",1,"Max Levchin,Novator,Shasta Ventures,Waypoint C...","cd730db2-a37f-0d5b-9b34-185edafe5017,1156fdfc-...",https://www.linkedin.com/in/alarik-myrin-035bab1/
5810,542067a6-bb15-eb5d-b01b-a383bb3df7f2,Zwift,"Health Care,Sports",1,"Max Levchin,Novator,Shasta Ventures,Waypoint C...","cd730db2-a37f-0d5b-9b34-185edafe5017,1156fdfc-...",https://www.linkedin.com/in/eric-min-82251b4/
5811,542067a6-bb15-eb5d-b01b-a383bb3df7f2,Zwift,"Health Care,Sports",1,"Max Levchin,Novator,Shasta Ventures,Waypoint C...","cd730db2-a37f-0d5b-9b34-185edafe5017,1156fdfc-...",http://www.linkedin.com/pub/meir-machlin/8/bb5...


In [301]:
# Load founder info 
def insert_new_cols(df):
  df.insert(0,'gender',pd.NA)
  df.insert(0,'age',pd.NA)
  df.insert(0,'past_roles',pd.NA)
  df.insert(0,'past_companies',pd.NA)
  df.insert(0,'education',pd.NA)
  df.insert(0,'degrees',pd.NA)

In [302]:
# function to update a column
def update_df(df,column_change,column_id,id,new_value):
  for index,element in enumerate(df[column_id]):
    if element == id:
      df.loc[index,column_change] = new_value

In [303]:
# function to update a column
def update_df_list(df,column_change,column_id,id,new_value):
  df[column_change] = df[column_change].astype(object)
  for index,element in enumerate(df[column_id]):
    if element == id:    
      df.at[index,column_change] = new_value

In [304]:
from pandas.core import series
import math

no_noEducation = 0
no_noDegree = 0
no_noEmployment = 0

# Update data frame
def update_all_cols(df):
  for url in df['founder_linkedin_url']:
    # check founder_url is not empty
    if isinstance(url,str):
      # Check url does not repeat: if repeat a series is returned
      if isinstance(success_enriched_linkedin_profiles.loc[url,'json_string'],pd.Series):
        profile_json = success_enriched_linkedin_profiles.loc[url,'json_string'][0]
      else:
        profile_json = success_enriched_linkedin_profiles.loc[url,'json_string']
      
      # check whether profile_json is NA
      if not pd.isna(profile_json):
        profile_json = json.loads(profile_json)

        # check 'data' exists
        if 'data' in profile_json:

          # check 'data' is not empty
          if len(profile_json['data']) != 0:

            # check 'age' exist
            if 'age' in profile_json['data'][0]:
              age = profile_json['data'][0]['age']
              update_df(df,'age','founder_linkedin_url',url,age) 
              # df.loc[url,'age'] = age
            else:
              pass
              # print('age not in')
              # print(url, profile_json['data'])
            
            # check 'gender' exist: note that 'gender' might have different keys
            if 'gender' in profile_json['data'][0]:
              if 'normalizedValue' in profile_json['data'][0]['gender']:
                gender = profile_json['data'][0]['gender']['normalizedValue']
                update_df(df,'gender','founder_linkedin_url',url,gender)
                # df.loc[url,'gender'] = gender
              elif 'str' in profile_json['data'][0]['gender']:
                gender = profile_json['data'][0]['gender']['str']
                update_df(df,'gender','founder_linkedin_url',url,gender)
                # df.loc[url,'gender'] = gender
              else:
                print(profile_json['data'][0]['gender'])
              
            # check 'education' exist
            if 'educations' in profile_json['data'][0]:
              educations = []
              degrees = []
              for education in profile_json['data'][0]['educations']:
                if 'institution'in education:
                  if 'degree' in education:
                    if ('name' in education['institution']) and ('name' in education['degree']):
                      educations.append(education['institution']['name'])
                      degrees.append(education['degree']['name'])
                  else:
                    if 'name' in education['institution']:
                      educations.append(education['institution']['name'])
              if educations:
                update_df_list(df_s1,'education','founder_linkedin_url',url,educations)
              if degrees:
                df_s1['degrees'] = df_s1['degrees'].astype(object)
                update_df_list(df_s1,'degrees','founder_linkedin_url',url,degrees)
              else:
                no_noDegree = no_noDegree + 1
            else:
              # print("'educations' don't exist")
              # print(f'this dude does not have education data: {url}')
              no_noEducation = no_noEducation + 1
            

            # check past companies exist
            if 'employments' in profile_json['data'][0]:
              employments = []
              roles = []
              for employment in profile_json['data'][0]['employments']:
                if 'employer' in employment:
                  if 'name' in employment['employer']:
                    employments.append(employment['employer']['name'])
                else:
                  pass
                  # print(employment)
                  # print(f'This dude has no employer: {url}')
                  # break
                if 'title' in employment:
                  roles.append(employment['title'])

              if employments:
                update_df_list(df_s1,'past_companies','founder_linkedin_url',url,employments)
              if roles:
                update_df_list(df_s1,'past_roles','founder_linkedin_url',url,roles)

            else:
              # pass
              print(profile_json['data'][0])
              print(f'No employments for this man: {url}')
              no_noEmployments = no_noEmployment + 1

          else:
            pass
            # print(url, profile_json['data'])
            # print("'data' is a empty list")  
        else:
          pass
          # print("'data' not in profile_json")
      else:
        pass
        # print('profile_json is NA')
      # try:
      #   profile_json = json.loads(profile_json)
      # except:
      #   print(pd.isna(success_enriched_linkedin_profiles.loc[url,'json_string']))
      #   print(url, profile_json)
      #   print('failed to load json')
    else:
      print('url not a string')

In [305]:
no_noEmployment

0

In [306]:
fail_enriched_linkedin_profiles.loc['http://www.linkedin.com/pub/pete-foley/3/753/a1a']['json_string']

'{"version": 1, "hits": 1, "results": 1, "kgversion": "285", "diffbot_type": "Person", "facet": false, "textFallback": false, "data": [{"gender": {"normalizedValue": "Male"}, "importance": 95.61087799072266, "origin": "wikidata.org/entity/Q100791648", "description": "Pete Foley is a CEO at ModelOp and is based in USA. He has experience at Yale Bulldogs men\'s basketball and has worked as CEO at PortAuthority Technologies, co-founder at Sysix Technology LLC, and CEO at RingCube Technologies. Pete attended Yale University between 1980 and 1984.", "profileChangeProbability": 0.6266520023345947, "educations": [{"institution": {"summary": "Undergraduate liberal arts college of Yale University", "image": "https://kg.diffbot.com/image/api/get?fetch=yes&url=g%3Cj7P0St0DnBJf.x0KwLZrUn.%5B%3CR0Aa4Hh%3B%5Bv738ZqOr7U%3Fav%40u%3E%7EO8Rbqh4W_JlxL%2F6%40nMj7Hqd.5fx", "types": ["Organization", "Corporation", "Company"], "name": "Yale University", "diffbotUri": "http://diffbot.com/entity/ED-_bsIN4MiS7b

In [307]:
df_s1.head()

,org_uuid,org_name,category_list,labels,investors_names,investors_uuids,founder_linkedin_url
0,a2323b6c-29b9-4750-905c-cdcbd9ce92b8,10X Banking,"Financial Services,Lending and Investments,Sof...",1,"Oliver Wyman,Ping An Global Voyager","88213ccc-71f8-1024-72cf-44e863af4993,c8cb5641-...",https://www.linkedin.com/in/antony-jenkins/
1,cf3b00cf-ffbf-0e4f-6427-a60e5060de39,10X Genomics,"Biotechnology,Health Care,Science and Engineering",1,Silicon Valley Bank,4eee5e60-3b06-6fe8-cc2c-29119a96051d,https://www.linkedin.com/in/ben-hindson-09739813/
2,cf3b00cf-ffbf-0e4f-6427-a60e5060de39,10X Genomics,"Biotechnology,Health Care,Science and Engineering",1,Silicon Valley Bank,4eee5e60-3b06-6fe8-cc2c-29119a96051d,https://www.linkedin.com/in/serge-saxonov-7b42...
3,0680987f-23cd-4fa5-850c-38326fffaa36,"111, Inc.",Health Care,1,Ivy Capital,f8eed790-8c07-4e49-c873-0f7245e393ff,https://www.linkedin.com/in/junling-liu-2b90724/
4,a79a6ae8-3c74-0db7-7e6b-b4f7662d37e8,17zuoye,"Education,Internet Services,Software",1,Shunwei Capital,24540ba2-35b0-a757-f75e-87565c50692e,https://www.linkedin.com/in/dun-xiao-3622629/


Combine companies together

In [308]:
# Combine companies together
concat_companies = pd.concat([df_s1,df_f1],ignore_index=True)
# concat_companies.to_excel(r'/content/gdrive/MyDrive/Colab Notebooks/weave/Datasets/concat_companies.xlsx', index=False)

In [309]:
concat_companies.head()

,org_uuid,org_name,category_list,labels,investors_names,investors_uuids,founder_linkedin_url
0,a2323b6c-29b9-4750-905c-cdcbd9ce92b8,10X Banking,"Financial Services,Lending and Investments,Sof...",1,"Oliver Wyman,Ping An Global Voyager","88213ccc-71f8-1024-72cf-44e863af4993,c8cb5641-...",https://www.linkedin.com/in/antony-jenkins/
1,cf3b00cf-ffbf-0e4f-6427-a60e5060de39,10X Genomics,"Biotechnology,Health Care,Science and Engineering",1,Silicon Valley Bank,4eee5e60-3b06-6fe8-cc2c-29119a96051d,https://www.linkedin.com/in/ben-hindson-09739813/
2,cf3b00cf-ffbf-0e4f-6427-a60e5060de39,10X Genomics,"Biotechnology,Health Care,Science and Engineering",1,Silicon Valley Bank,4eee5e60-3b06-6fe8-cc2c-29119a96051d,https://www.linkedin.com/in/serge-saxonov-7b42...
3,0680987f-23cd-4fa5-850c-38326fffaa36,"111, Inc.",Health Care,1,Ivy Capital,f8eed790-8c07-4e49-c873-0f7245e393ff,https://www.linkedin.com/in/junling-liu-2b90724/
4,a79a6ae8-3c74-0db7-7e6b-b4f7662d37e8,17zuoye,"Education,Internet Services,Software",1,Shunwei Capital,24540ba2-35b0-a757-f75e-87565c50692e,https://www.linkedin.com/in/dun-xiao-3622629/


Create edge data

In [310]:
# create a dictionary with investor_uuid as key and a list of org_uuid as value
edges_base = {}

for org_index,org in enumerate(concat_companies['org_uuid']):
  # get a list of investor uuids for each company
  investors = concat_companies.loc[org_index,'investors_uuids']
  investors = investors.split(',')

  # put org_uuid for each investors_uuids
  for investor in investors:
    if edges_base.get(investor) == None: 
      edges_base[investor] = []
    if org not in edges_base[investor]:
      edges_base[investor].append(org)
    
    

print(f"The number of investors is {len(edges_base)}.")


The number of investors is 10622.


In [311]:
# with open('/content/gdrive/MyDrive/Colab Notebooks/weave/KnowledgeBase/edges_base.pickle', 'wb') as file:
#   pickle.dump(edges_base, file)